In [1]:
# Configuración básica del proyecto
PROJECT_NAME = 'Monte Carlo Dropout'

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from classes.DataHandler import DataHandler
from classes.MonteCarloDropoutNetwork_opt import MonteCarloDropoutNetwork_opt

dh = DataHandler()
DEVICE = 'cuda'
dh.DEVICE = DEVICE

dh.set_targets(['Mo_cumulative'])
table_unified, vars_desc = dh.load_data()
inputs, targets = dh.preprocess_data(vars_desc, table_unified)
X, y, scales = dh.normalize_data(inputs, targets)

2025-05-17 12:28:38,518 - INFO - Configuración de logging inicializada correctamente.
2025-05-17 12:28:55,454 - INFO - Columnas objetivo actualizadas: ['Mo_cumulative']
2025-05-17 12:28:55,455 - INFO - Inicio de la carga de datos.
2025-05-17 12:28:55,460 - INFO - Tabla Tabla_Unificada cargada correctamente.
2025-05-17 12:28:55,461 - INFO - Tabla Variables_Description cargada correctamente.
2025-05-17 12:28:55,466 - INFO - Conexión cerrada.
2025-05-17 12:28:55,454 - INFO - Columnas objetivo actualizadas: ['Mo_cumulative']
2025-05-17 12:28:55,455 - INFO - Inicio de la carga de datos.
2025-05-17 12:28:55,460 - INFO - Tabla Tabla_Unificada cargada correctamente.
2025-05-17 12:28:55,461 - INFO - Tabla Variables_Description cargada correctamente.
2025-05-17 12:28:55,466 - INFO - Conexión cerrada.
2025-05-17 12:28:55,467 - INFO - Datos cargados exitosamente.
2025-05-17 12:28:55,467 - INFO - Inicio del preprocesamiento de datos.
2025-05-17 12:28:55,467 - INFO - Datos cargados exitosamente.
202

🔗 Conexión establecida con data/Tronaduras_vs_Sismicidad.db
✅ Datos leídos desde 'Processed_Data.Tabla_Unificada' y convertidos a DataFrame.
✅ Datos leídos desde 'Raw_Data.Variables_Description' y convertidos a DataFrame.
🔌 Conexión cerrada manualmente.


In [2]:
# Convertir X y y a tensores de PyTorch y moverlos a la GPU
X = torch.tensor(X, dtype=torch.float32).to(dh.DEVICE)
y = torch.tensor(y, dtype=torch.float32).to(dh.DEVICE)

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(
    X.cpu().numpy(), y.cpu().numpy(), test_size=0.2, random_state=42
)

# Convertir los conjuntos a tensores y moverlos a la GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(dh.DEVICE)
X_val = torch.tensor(X_val, dtype=torch.float32).to(dh.DEVICE)
y_train = torch.tensor(y_train, dtype=torch.float32).to(dh.DEVICE)
y_val = torch.tensor(y_val, dtype=torch.float32).to(dh.DEVICE)

input_dim = X_train.shape[1]
hidden_dim = 512
dropout_prob = 0.2
show_info = 'True'

mcd = MonteCarloDropoutNetwork_opt(input_dim, hidden_dim, DEVICE, dropout_prob, show_info=show_info)
model = mcd.train_model(X_train, y_train, X_val, y_val, num_epochs=5000, lr=0.001)

[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 0 📉 train loss: 0.1343 📊 val loss: 0.0439
[2025-05-17 12:29:21] 🔄 [Época 0] 🧠 Pérdida entrenamiento: 0.1343 | Validación: 0.0439 🚀
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 1 📉 train loss: 0.0518 📊 val loss: 0.0265
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 6 📉 train loss: 0.0414 📊 val loss: 0.0259
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 7 📉 train loss: 0.0275 📊 val loss: 0.0181
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 14 📉 train loss: 0.0271 📊 val loss: 0.0165
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 15 📉 train loss: 0.0217 📊 val loss: 0.0159
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 23 📉 train loss: 0.0186 📊 val loss: 0.0156
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 31 📉 train loss: 0.0196 📊 val loss: 0.0156
[2025-05-17 12:29:21] ✨ Nueva mejor red encontrada en época 38 📉 train loss: 0.0181 📊 val l

In [3]:
# Predicción MC Dropout y desnormalización
y_pred_train, y_pred_train_std = mcd.predict_mc_dropout(X_train, num_samples=100)
y_pred_val, y_pred_val_std = mcd.predict_mc_dropout(X_val, num_samples=100)

y_train_np = y_train.cpu().numpy()
y_val_np = y_val.cpu().numpy()
y_train_mo = dh.denormalize_value(scales, y_train_np, 'Mo_cumulative')
y_pred_train_mo = dh.denormalize_value(scales, y_pred_train, 'Mo_cumulative')
y_val_mo = dh.denormalize_value(scales, y_val_np, 'Mo_cumulative')
y_pred_val_mo = dh.denormalize_value(scales, y_pred_val, 'Mo_cumulative')

In [4]:
# Graficar resultados con intervalo de confianza
dh.plot_with_confidence_interval(
    (y_train_mo, 'Valores Reales'),
    (y_pred_train_mo, y_pred_train_mo - 1.96 * y_pred_train_std, y_pred_train_mo + 1.96 * y_pred_train_std, 'MCD - Predicciones'),
    title='MCD: Predicciones con Intervalo (Entrenamiento)',
    size=(1200, 400)
)

dh.plot_with_confidence_interval(
    (y_val_mo, 'Valores Reales'),
    (y_pred_val_mo, y_pred_val_mo - 1.96 * y_pred_val_std, y_pred_val_mo + 1.96 * y_pred_val_std, 'MCD - Predicciones'),
    title='MCD: Predicciones con Intervalo (Validación)',
    size=(1200, 400)
)

In [5]:
# Evaluación
dh.evaluar_metricas(y_train_mo, y_pred_train_mo, 'MCD - Entrenamiento')
dh.evaluar_metricas(y_val_mo, y_pred_val_mo, 'MCD - Validación')


📊 Métricas para MCD - Entrenamiento:
   MAE  : 0.0250
   RMSE : 0.0308
   R²   : 0.9944

📊 Métricas para MCD - Validación:
   MAE  : 0.2619
   RMSE : 0.3346
   R²   : 0.3219


{'MAE': 0.261852115985948,
 'RMSE': np.float64(0.3346290799630877),
 'R2': 0.32189855781050125}

In [6]:
# Gráficos de regresión
dh.plot_regression(y_train_mo, y_pred_train_mo, title='Regresión MCD - Entrenamiento', size=(1200, 400))
dh.plot_regression(y_val_mo, y_pred_val_mo, title='Regresión MCD - Validación', size=(1200, 400))